## Step 1 : A) Examine the dataset

##### Load the data and understand its structure. Identify each column, its data type, and split data accoording to its prefix (C, S, Q, target). The dataset includes various data types such as numerical, categorical, and boolean, so I’ll need to handle each type appropriately.

In [30]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [32]:
inputs = pd.read_csv('../data/raw/inputs.csv', sep=';')
inputs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85529 entries, 0 to 85528
Columns: 109 entries, ID to Q75
dtypes: float64(40), int64(18), object(51)
memory usage: 71.1+ MB


In [33]:
households_columns = ["ID", "COD_INSEE", "COD_IRIS"]
clients_columns = ["ID"] + [f"C{i}" for i in range(1, 20)]
site_columns = ["ID"] + [f"S{i}" for i in range(1, 13)]
questions_columns = ["ID"] + [f"Q{i}" for i in range(1, 76)]

In [34]:
inputs[households_columns].to_csv('../data/raw/households.csv', index=False)
inputs[clients_columns].to_csv('../data/raw/clients.csv', index=False)
inputs[site_columns].to_csv('../data/raw/site.csv', index=False)
inputs[questions_columns].to_csv('../data/raw/questions.csv', index=False)

### Clients data

In [35]:
clients = pd.read_csv('../data/raw/clients.csv')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85529 entries, 0 to 85528
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      85529 non-null  int64 
 1   C1      85529 non-null  object
 2   C2      85529 non-null  object
 3   C3      85529 non-null  object
 4   C4      85529 non-null  object
 5   C5      85529 non-null  int64 
 6   C6      85529 non-null  int64 
 7   C7      85529 non-null  int64 
 8   C8      85529 non-null  object
 9   C9      85529 non-null  object
 10  C10     85529 non-null  int64 
 11  C11     85529 non-null  int64 
 12  C12     85529 non-null  object
 13  C13     85529 non-null  object
 14  C14     85529 non-null  object
 15  C15     85529 non-null  int64 
 16  C16     85529 non-null  int64 
 17  C17     85529 non-null  int64 
 18  C18     85529 non-null  int64 
 19  C19     85529 non-null  int64 
dtypes: int64(11), object(9)
memory usage: 13.1+ MB


In [15]:
clients.head(10)

,ID,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19
0,0,EA,RA,C,IB,0,0,0,YA,RA,8,2286,OB,RC,OB,2,214,212,16,7
1,1,EA,RA,C,IA,0,0,0,YB,RA,166,2220,OB,RC,OB,3,511,414,15,19
2,2,EA,RC,B,IA,0,0,0,YB,RA,17,2265,OC,R7,OA,4,407,405,17,8
3,3,EA,RA,C,IB,0,0,0,YB,RA,16,2250,OA,RC,OB,3,712,410,16,12
4,4,EA,RA,B,IB,0,0,0,YA,RA,23,2305,OB,RC,OA,1,206,204,17,4
5,5,EA,RA,B,IB,0,0,0,YB,RA,112,2260,OB,RC,OA,2,417,415,15,13
6,6,EA,RA,B,IB,0,1,0,YB,RA,14,2230,OA,RC,OB,2,609,407,16,12
7,7,EA,RA,C,IB,0,0,0,YB,RA,42,2238,OB,RC,OA,5,108,106,16,4
8,8,EA,RA,C,IB,0,0,0,YB,RA,10,2208,OB,RC,OB,4,510,413,15,12
9,9,EA,RA,C,IB,0,0,0,YB,RA,24,2297,OB,RC,OB,2,111,109,17,4


In [22]:
clients.isna().sum()

ID     0
C1     0
C2     0
C3     0
C4     0
C5     0
C6     0
C7     0
C8     0
C9     0
C10    0
C11    0
C12    0
C13    0
C14    0
C15    0
C16    0
C17    0
C18    0
C19    0
dtype: int64

The client data have no missing values (no NaN values).

In [36]:
objects = clients.select_dtypes(include=['object']).columns.to_list()
ints = clients.select_dtypes(include=['int64']).columns.to_list()
print("object columns are :", objects)
print("ints columns are :", ints)

object columns are : ['C1', 'C2', 'C3', 'C4', 'C8', 'C9', 'C12', 'C13', 'C14']
ints columns are : ['ID', 'C5', 'C6', 'C7', 'C10', 'C11', 'C15', 'C16', 'C17', 'C18', 'C19']


In [37]:
for column in objects:
    print(f"Unique values in column '{column}':")
    print(clients[column].unique())
    print("-" * 40)

Unique values in column 'C1':
['EA']
----------------------------------------
Unique values in column 'C2':
['RA' 'RC' 'RB' 'R7']
----------------------------------------
Unique values in column 'C3':
['C' 'B' 'A' 'E' 'D' 'F' 'G' 'I' 'H']
----------------------------------------
Unique values in column 'C4':
['IB' 'IA']
----------------------------------------
Unique values in column 'C8':
['YA' 'YB']
----------------------------------------
Unique values in column 'C9':
['RA' 'RB']
----------------------------------------
Unique values in column 'C12':
['OB' 'OC' 'OA' 'O7']
----------------------------------------
Unique values in column 'C13':
['RC' 'R7' 'RD' 'RA' 'RB']
----------------------------------------
Unique values in column 'C14':
['OB' 'OA']
----------------------------------------


We drop C1 since it is a constant. For C2, C12 and C13, we can use one-hot encoding. For C3, we can use ordinal encoding. For the rest of the columns, we can use label encoding. 

In [38]:
clients.drop('C1', axis=1, inplace=True)
clients = pd.get_dummies(clients, columns=['C2', 'C12', 'C13'], drop_first=False)

label_cols = ['C4', 'C8', 'C9', 'C14']
label_encoder = LabelEncoder()
for col in label_cols:
    clients[col] = label_encoder.fit_transform(clients[col])

ordinal_encoder = OrdinalEncoder()
clients['C3'] = ordinal_encoder.fit_transform(clients[['C3']])

In [ ]:
clients[clients.select_dtypes(include=['float64', 'bool']).columns.to_list()] = clients[clients.select_dtypes(include=['float64', 'bool']).columns.to_list()].astype('int64')
clients[[column for column in clients.columns if column not in ints]].head(10)

In [46]:
for column in ints[1:]:
    print(f"Unique values in column '{column}':")
    print(clients[column].unique())
    print("-" * 40)

Unique values in column 'C5':
[0 1]
----------------------------------------
Unique values in column 'C6':
[0 1]
----------------------------------------
Unique values in column 'C7':
[0 1]
----------------------------------------
Unique values in column 'C10':
[   8  166   17   16   23  112   14   42   10   24   30   76   52   33
   38  128   22   13   18  116   39    5   37   41    7   93   96  176
   31   12   28   46   25    2   34    3   11    6    4    1   36   66
   56   64   86    9  104  216  101   40   26   32   35   29   43  115
   51   15   19   79   21   48   27  109   20  316  236   97   44  106
  250   95   53   69   45   47   49  126   63   73   61   87   62   81
   65   54   68  206   57   83  140   80  147    0  110  146  137  167
   88   94   50  261   59  196   78   60  111  516  186   71  231  136
  227  120   91  156  190   58  193  616   67  706  232  131  401   75
  105  134  168  229   74  328  102  262  266  149  132   84  125  164
  222  243  416  201  114   

In [47]:
clients[['C10', 'C11']] = clients[['C10', 'C11']].astype('float')
clients[['C3', 'C15', 'C16', 'C17', 'C18', 'C19']] = clients[['C3', 'C15', 'C16', 'C17', 'C18', 'C19']].astype('category')

In [60]:
clients.columns

Index(['ID', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19'],
      dtype='object')

In [61]:
clients.to_csv('../data/processed/clients.csv', index=False)

#### Site Data

In [49]:
site = pd.read_csv('../data/raw/site.csv')
site.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85529 entries, 0 to 85528
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      85529 non-null  int64  
 1   S1      58668 non-null  float64
 2   S2      85529 non-null  int64  
 3   S3      85529 non-null  object 
 4   S4      85529 non-null  object 
 5   S5      85529 non-null  object 
 6   S6      58668 non-null  object 
 7   S7      58668 non-null  object 
 8   S8      85529 non-null  int64  
 9   S9      85529 non-null  int64  
 10  S10     85529 non-null  int64  
 11  S11     85529 non-null  int64  
 12  S12     85529 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 8.5+ MB


In [50]:
site.head(10)

,ID,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12
0,0,1.0,0,2013-11-15,2014-02-22,2013-09-08,2013-09-07,2013-09-07,6,8,1,2,1
1,1,3.0,2,2013-11-15,2014-08-07,2013-04-18,2013-09-14,2014-03-21,11,21,4,1,1
2,2,NaN,0,2013-11-15,2013-11-05,2013-11-05,NaN,NaN,1,3,1,1,0
3,3,4.0,1,2013-11-15,2014-05-30,2013-07-03,2014-01-30,2014-05-18,10,13,4,0,5
4,4,NaN,1,2013-11-15,2013-09-22,2013-09-22,NaN,NaN,1,1,1,0,0
5,5,3.0,1,2013-11-15,2014-08-01,2013-08-23,2014-04-03,2014-05-29,6,8,0,1,1
6,6,NaN,1,2013-11-15,2014-03-16,2013-10-21,NaN,NaN,5,3,0,1,1
7,7,2.0,0,2013-11-15,2014-06-14,2014-01-24,2014-01-23,2014-06-13,6,8,6,0,1
8,8,2.0,1,2014-03-13,2014-07-02,2014-04-01,2014-03-31,2014-07-01,2,6,2,1,0
9,9,1.0,0,2013-11-15,2014-05-20,2013-09-09,2013-09-08,2013-09-08,8,10,2,2,1


In [51]:
site.isna().sum()

ID         0
S1     26861
S2         0
S3         0
S4         0
S5         0
S6     26861
S7     26861
S8         0
S9         0
S10        0
S11        0
S12        0
dtype: int64

In [52]:
site = site.drop(columns=['S6', 'S7'])

In [53]:
objects = site.select_dtypes(include=['object']).columns.to_list()
ints = site.select_dtypes(include=['int64']).columns.to_list()
floats = site.select_dtypes(include=['float64']).columns.to_list()
print("object columns are :", objects)
print("ints columns are :", ints)
print("floats columns are :", floats)

object columns are : ['S3', 'S4', 'S5']
ints columns are : ['ID', 'S2', 'S8', 'S9', 'S10', 'S11', 'S12']
floats columns are : ['S1']


In [57]:
for column in ints[1:]:
    print(f"Unique values in column '{column}':")
    print(site[column].unique())
    print("-" * 40)

Unique values in column 'S2':
[ 0  2  1  4  3  5  6 14  8  7 15 12 16 26 10  9 11 24 20 19 17 18]
----------------------------------------
Unique values in column 'S8':
[  6  11   1  10   5   2   8  14   4   3  18  12   9  22  15  52  66  27
  34  24   7  13  28  16  17  69  20  23  25  49  21  29  26  33  58  30
  36  63  75  56  19  35  39  60  37  65  38  32  73  50  47  68 135  41
  54  85  31  86  43  51  44  40 138  67  53 128  46  64  61 136  42 129
  48  55 164 163 106  59  99 170  82 125 110  57  97  96  45  72  79  98
  76  62 116  83 207 194  91 130 133  88  84  71 108 178 117  92 365  87
 111 123 115 344  89  74 204 109 161 254 112 107  81  70  80  95 104 120
 298  94 144 150 225 361 218  77  78 148 145  90 158 174 137 100 105 209
 286 101 156  93 103 139 147 142 188 102 122 272 127 172 241 124 159 196
 411 118 184 197 412 175 326 278 132 140 154 141 114 232 364 134 146 233
 316 169 162 182 191 212 221 220 173 113 119 152 167 222]
----------------------------------------
Un

In [58]:
site[['S8', 'S9', 'S10']] = site[['S8', 'S9', 'S10']].astype('float')
site[['S2', 'S11', 'S12']] = site[['S2', 'S11', 'S12']].astype('category')

In [59]:
for column in floats:
    print(f"Unique values in column '{column}':")
    print(site[column].unique())
    print("-" * 40)

Unique values in column 'S1':
[  1.   3.  nan   4.   2.   6.   9.  14.   8.   5.   7.  37.  25.  20.
  10.  18.  15.  11.  22.  19.  16.  31.  12.  24. 108.  43.  39.  30.
  13.  28.  27.  21.  56.  93.  51.  29.  23.  44.  47.  17.  35.  38.
  80.  42.  40.  36.  66.  26.  86.  55.  41.  48. 178.  72.  50.  52.
  68.  79.  73.  32.  34.  85.  49.  33.  46.  84. 183. 137.  53.  70.
  78.  57.  67. 151.  97.  45.  58.  98. 139. 201.  96.  63.  54. 275.
 116. 118. 100. 218.  61.  62.  92.  99. 135. 109. 177.  59. 169. 233.
  74.  87.  76.  71.  81.  82. 129. 103.  64. 113. 121. 110. 144. 162.
  91. 180.  95. 128. 149.  69.  83. 182. 157. 326. 107. 223. 120.  65.
 123.  60. 112.  94.  89. 126. 117. 148. 167. 114. 146. 106. 216. 239.
 221. 153.  75. 124. 138. 172.  90. 205.]
----------------------------------------


In [62]:
site.columns

Index(['ID', 'S1', 'S2', 'S3', 'S4', 'S5', 'S8', 'S9', 'S10', 'S11', 'S12'], dtype='object')

In [64]:
site.to_csv('../data/processed/site.csv', index=False)

#### Questions data

In [216]:
questions = pd.read_csv('../data/raw/questions.csv')
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85529 entries, 0 to 85528
Data columns (total 76 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      85529 non-null  int64  
 1   Q1      85529 non-null  object 
 2   Q2      85521 non-null  object 
 3   Q3      85527 non-null  object 
 4   Q4      85527 non-null  float64
 5   Q5      85529 non-null  int64  
 6   Q6      27534 non-null  float64
 7   Q7      27672 non-null  object 
 8   Q8      84791 non-null  float64
 9   Q9      84777 non-null  float64
 10  Q10     85528 non-null  object 
 11  Q11     84751 non-null  object 
 12  Q12     85015 non-null  object 
 13  Q13     84507 non-null  float64
 14  Q14     83950 non-null  float64
 15  Q15     20062 non-null  float64
 16  Q16     84789 non-null  object 
 17  Q17     8756 non-null   float64
 18  Q18     8507 non-null   float64
 19  Q19     85492 non-null  float64
 20  Q20     84765 non-null  float64
 21  Q21     84801 non-null  float64
 22

In [217]:
objects = questions.select_dtypes(include=['object']).columns.to_list()
ints = questions.select_dtypes(include=['int64']).columns.to_list()
floats = questions.select_dtypes(include=['float64']).columns.to_list()
print("object columns are :", objects)
print("ints columns are :", ints)
print("floats columns are :", floats)

object columns are : ['Q1', 'Q2', 'Q3', 'Q7', 'Q10', 'Q11', 'Q12', 'Q16', 'Q28', 'Q29', 'Q32', 'Q34', 'Q36', 'Q39', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74', 'Q75']
ints columns are : ['ID', 'Q5']
floats columns are : ['Q4', 'Q6', 'Q8', 'Q9', 'Q13', 'Q14', 'Q15', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q30', 'Q31', 'Q33', 'Q35', 'Q37', 'Q38', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51', 'Q52']


In [218]:
threshold = 10
columns_to_drop = questions.columns[(100 * questions.isna().sum() / len(questions)) > threshold]
questions = questions.drop(columns=columns_to_drop)

In [219]:
objects = questions.select_dtypes(include=['object']).columns.to_list()

In [220]:
categorical = ['Q1', 'Q2', 'Q3', 'Q10', 'Q11', 'Q12', 'Q16', 'Q28', 'Q29', 'Q32', 'Q34', 'Q36']
boolean = ['Q53', 'Q58', 'Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68', 'Q69', 'Q70', 'Q71', 'Q72']

In [221]:
threshold = 1 
for column in questions[categorical].columns:
    value_counts = (questions[column].value_counts(normalize=True) * 100)
    rare_values = value_counts[value_counts < threshold].index.tolist()
    
    if rare_values:
        print(f"Column '{column}':")
        print(f"Rare values (less than {threshold}%): {rare_values}")
        print(f"Unique values : {questions[column].unique()}")
        
        questions[column] = questions[column].replace(rare_values, "Rare")
        
        print(f"Unique values after replacing them: {questions[column].unique()}")
        print("-" * 40)

Column 'Q1':
Rare values (less than 1%): ['H7', 'HB']
Unique values : ['HA' 'HC' 'H7' 'HB']
Unique values after replacing them: ['HA' 'HC' 'Rare']
----------------------------------------
Column 'Q2':
Rare values (less than 1%): ['Z7']
Unique values : ['ZA' 'ZB' 'ZC' 'Z7' nan]
Unique values after replacing them: ['ZA' 'ZB' 'ZC' 'Rare' nan]
----------------------------------------
Column 'Q3':
Rare values (less than 1%): ['U7', 'UB']
Unique values : ['UC' 'UA' 'UB' 'U7' nan]
Unique values after replacing them: ['UC' 'UA' 'Rare' nan]
----------------------------------------
Column 'Q10':
Rare values (less than 1%): ['P7', 'PE', 'PB']
Unique values : ['PC' 'PD' 'PA' 'P7' 'PB' 'PE' nan]
Unique values after replacing them: ['PC' 'PD' 'PA' 'Rare' nan]
----------------------------------------
Column 'Q11':
Rare values (less than 1%): ['DA', 'DH', 'D5', 'DC', 'D9', 'D8', 'DP', 'DN', 'D0', 'DE', 'DT', 'D4', 'DV', 'DR']
Unique values : ['D7' 'DS' 'D3' 'DB' 'DF' 'DO' 'D1' 'DU' 'DQ' 'DA' 'DD' 'DI'

In [222]:
questions = pd.get_dummies(questions, columns=categorical, drop_first=False)

In [223]:
for column in boolean:
    print(f"Unique values in column '{column}':")
    print(questions[column].unique())
    print("-" * 40)

Unique values in column 'Q53':
['False' 'true' 'false' 'True' nan 'true ' False True]
----------------------------------------
Unique values in column 'Q58':
['false' 'true' nan 'true ']
----------------------------------------
Unique values in column 'Q59':
['false' 'true' nan 'true ']
----------------------------------------
Unique values in column 'Q60':
['false' 'true' nan 'False' 'true ' 'True' False True]
----------------------------------------
Unique values in column 'Q61':
['false' 'true' nan 'true ']
----------------------------------------
Unique values in column 'Q62':
['false' 'true' 'true ' nan]
----------------------------------------
Unique values in column 'Q63':
['false' 'False' 'true' nan 'True' 'true ' False True]
----------------------------------------
Unique values in column 'Q63':
['false' 'False' 'true' nan 'True' 'true ' False True]
----------------------------------------
Unique values in column 'Q64':
['false' 'true' nan 'true ']
----------------------------

In [224]:
mapping = {'true': True, 'false': False}
for column in boolean:
    questions[column] = questions[column].fillna('unknown').astype(str).str.strip().str.lower().map(mapping)

In [225]:
label_encoder = LabelEncoder()
for col in boolean:
    questions[col] = label_encoder.fit_transform(questions[col])

In [226]:
questions[boolean].head(10)

,Q53,Q58,Q59,Q60,Q61,Q62,Q63,Q63,Q64,Q65,Q66,Q67,Q68,Q69,Q70,Q71,Q72
0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0
1,1,1,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
6,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
9,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0


In [227]:
floats = questions.select_dtypes(include=['float64']).columns.to_list()
print("floats columns are :", floats)

for column in floats:
    print(f"Unique values in column '{column}':")
    print(questions[column].unique())
    print("-" * 40)

floats columns are : ['Q4', 'Q8', 'Q9', 'Q13', 'Q14', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q27', 'Q30', 'Q31', 'Q33', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 'Q51']
Unique values in column 'Q4':
[8.000e+00 1.660e+02 1.700e+01 1.600e+01 2.300e+01 1.120e+02 1.400e+01
 6.600e+01 1.000e+01 2.400e+01 3.000e+01 7.600e+01 4.200e+01 5.200e+01
 3.300e+01 3.800e+01 1.280e+02 2.200e+01 1.300e+01 1.800e+01 1.160e+02
 3.900e+01 5.000e+00 3.700e+01 4.100e+01 7.000e+00 9.300e+01 9.600e+01
 1.760e+02 3.100e+01 1.200e+01 2.800e+01 4.600e+01 2.500e+01 2.000e+00
 3.400e+01 3.000e+00 1.100e+01 6.000e+00 4.000e+00 1.000e+00 3.600e+01
 5.600e+01 6.400e+01 8.600e+01 9.000e+00 1.040e+02 2.160e+02 1.010e+02
 4.000e+01 2.600e+01 3.200e+01 3.500e+01 2.900e+01 4.300e+01 1.150e+02
 5.100e+01 1.500e+01 1.900e+01 7.900e+01 2.100e+01 4.800e+01 2.700e+01
 1.090e+02 2.000e+01 3.160e+02 2.360e+02 9.700e+01 4.400e+01 1.060e+02
 2.500e+02 9.500e+01 5.300e+01 6.900e+01 4.500e+01 4

In [228]:
to_one_hot = ['Q8', 'Q21', 'Q23', 'Q24', 'Q25', 'Q27', 'Q41' ,'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q49', 'Q50']
to_int = ['Q19', 'Q31', 'Q48']

In [229]:
most_common_values = questions[to_int].mode().iloc[0].to_dict()
questions[to_int] = questions[to_int].apply(lambda x: x.fillna(most_common_values[x.name]))
questions[to_int] = questions[to_int].astype('int64')

In [230]:
threshold = 1 
for column in questions[to_one_hot].columns:
    value_counts = (questions[column].value_counts(normalize=True) * 100)
    rare_values = value_counts[value_counts < threshold].index.tolist()
    
    if rare_values:
        print(f"Column '{column}':")
        print(f"Rare values (less than {threshold}%): {rare_values}")
        print(f"Unique values : {questions[column].unique()}")
        
        questions[column] = questions[column].replace(rare_values, "Rare")
        
        print(f"Unique values after replacing them: {questions[column].unique()}")
        print("-" * 40)

Column 'Q8':
Rare values (less than 1%): [3.0, 1.0, 5.0]
Unique values : [ 2.  0.  4. nan  1.  3.  5.]
Unique values after replacing them: [2.0 0.0 4.0 nan 'Rare']
----------------------------------------
Column 'Q21':
Rare values (less than 1%): [2.0]
Unique values : [ 1.  0. nan  2.]
Unique values after replacing them: [1.0 0.0 nan 'Rare']
----------------------------------------
Column 'Q23':
Rare values (less than 1%): [2.0]
Unique values : [ 1.  0. nan  2.]
Unique values after replacing them: [1.0 0.0 nan 'Rare']
----------------------------------------
Column 'Q24':
Rare values (less than 1%): [2.0]
Unique values : [ 0.  1. nan  2.]
Unique values after replacing them: [0.0 1.0 nan 'Rare']
----------------------------------------
Column 'Q25':
Rare values (less than 1%): [2.0]
Unique values : [ 1.  0. nan  2.]
Unique values after replacing them: [1.0 0.0 nan 'Rare']
----------------------------------------
Column 'Q27':
Rare values (less than 1%): [2.0]
Unique values : [ 0.  1. na

In [231]:
questions = pd.get_dummies(questions, columns=to_one_hot, drop_first=True)

In [232]:
booleans = questions.select_dtypes(include=['bool']).columns.to_list()
int_32 = questions.select_dtypes(include=['int32']).columns.to_list()
questions[booleans] = questions[booleans].astype('int64')
questions[int_32] = questions[int_32].astype('int64')

In [233]:
questions.to_csv('../data/processed/questions.csv', index=False)